## Creating Staging Tables from GCS

In [7]:
staging_dataset_id = "MIT_staging"
!bq --location=US mk --dataset {staging_dataset_id}

Dataset 'sound-cider-252823:MIT_staging' successfully created.


### President Staging

In [8]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {staging_dataset_id}.President_Elections \
'gs://cs327efall2019_electionresults/President Data.csv'

Waiting on bqjob_r4bd0d62fb671af53_0000016e6cd8e4c4_1 ... (2s) Current status: DONE   


In [11]:
%%bigquery
select count(*)
from MIT_staging.President_Elections

,f0_
0,3740


### Senate Staging

In [9]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {staging_dataset_id}.Senate_Elections \
'gs://cs327efall2019_electionresults/Senate.csv'

Waiting on bqjob_r285ac7e062349a2a_0000016e6cdb919d_1 ... (1s) Current status: DONE   


In [12]:
%%bigquery
select count(*)
from MIT_staging.Senate_Elections

,f0_
0,3421


### House Staging

In [13]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {staging_dataset_id}.House_Elections \
'gs://cs327efall2019_electionresults/house.csv'

Waiting on bqjob_r4d9dbe358db71a12_0000016e6ce14ed7_1 ... (2s) Current status: DONE   


In [13]:
%%bigquery
select count(*)
from MIT_staging.House_Elections

,f0_
0,29636


## Creating Intermediate Tables

In [14]:
intermediate_dataset_id = "MIT_intermediate"
!bq --location=US mk --dataset {intermediate_dataset_id}

Dataset 'sound-cider-252823:MIT_intermediate' successfully created.


In [16]:
%%bigquery
create or replace table MIT_intermediate.Elections as
((select year as Year, state_po as State, "House" as Type, District, stage as Stage, runoff as Runoff, special as Special, totalvotes as Total_Votes
from MIT_staging.House_Elections)
union all 
(select year as Year, state_po as State, "President" as Type, null as District, null as Stage, null as Runoff, null as Speacial, totalvotes as Total_Votes
from MIT_staging.President_Elections)
union all 
(select year as Year, state_po as State, "Senate" as Type, null as District, stage as Stage, null as Runoff, special as Speacial, totalvotes as Total_Votes
from MIT_staging.Senate_Elections))

""


In [5]:
%%bigquery
create or replace table MIT_intermediate.Elections_with_PK as
(
select 
CONCAT(cast(Year as string), State, Type, if(District is Null, "NA", cast(District as string)), if(Stage is Null, "NA", Stage)) as Election_ID, Year, State, Type, District, Stage, Runoff, Special, Total_Votes
from MIT_intermediate.Elections
)

""


In [ ]:
#Remove dupicates code below

In [10]:
%%bigquery
create or replace table MIT_intermediate.Elections_Distinct_PK as
(select distinct *
from MIT_intermediate.Elections_with_PK)

""


In [17]:
%%bigquery
create or replace table MIT_intermediate.Elections_Casted as
(select Election_ID, Year, State, Type, District, Stage, cast(if( Runoff = "NA", null, Runoff) as Bool) as Runoff, Special, cast(TRUNC( Total_Votes ) as int64) as Total_Votes
from MIT_intermediate.Elections_Distinct_PK)

""


## Creating Modeled Tables

In [18]:
modeled_dataset_id = "MIT_modeled"
!bq --location=US mk --dataset {modeled_dataset_id}

Dataset 'sound-cider-252823:MIT_modeled' successfully created.


In [19]:
%%bigquery
create or replace table MIT_modeled.Elections as
(select Election_ID, Year, State, Type, District, Stage, cast(if( Runoff = "NA", null, Runoff) as Bool) as Runoff, Special, cast(TRUNC( Total_Votes ) as int64) as Total_Votes
from MIT_intermediate.Elections_Distinct_PK)

""


Note that the below creation of the Runs table doesnot create a primary key for that table. This is because this requires a nontrivial linking of candidate names between the two datasets. We lack necessary knowledge to complete such linking.

In [21]:
%%bigquery
create or replace table MIT_modeled.Runs as
(select Candidate, Candidate_Votes, CONCAT(cast(Year as string), State, Type, if(District is Null, "NA", cast(District as string)), if(Stage is Null, "NA", cast(Stage as string))) as Election_ID
from(
(select candidate as Candidate, candidatevotes as Candidate_Votes, year as Year, state_po as State, "House" as Type, District, stage as Stage, totalvotes as Total_Votes
from MIT_staging.House_Elections)
union all 
(select candidate as Candidate, candidatevotes as Candidate_Votes, year as Year, state_po as State, "President" as Type, null as District, null as Stage, totalvotes as Total_Votes
from MIT_staging.President_Elections)
union all 
(select candidate as Candidate, candidatevotes as Candidate_Votes, year as Year, state_po as State, "Senate" as Type, null as District, state as Stage, totalvotes as Total_Votes
from MIT_staging.Senate_Elections)) where Candidate != 'NA')

""
